<a href="https://colab.research.google.com/github/JonatanSiracusa/download-historical-series/blob/main/downloading_hist_prices.ipynb" target="_blank">
	<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Downloading Historical Prices

This program retrieves financial asset prices from Yahoo Finance, calculates simple and continuous returns, and determines volatility based on adjusted prices.  
Additionally, it allows exporting the processed data for further analysis.

Before running the program, users must configure the necessary parameters, which are detailed below.


## Project Process Overview

The following key steps will be followed in the implementation of this ***Project***:

1. **Kick-off**: Importing libraries, setting up variables, and defining essential functions.

2. **Data Loading**: Retrieving the required datasets.

3. **Data Cleaning**: Removing inconsistencies and handling missing values.

4. **Data Transformation**: Structuring data and performing calculations for further analysis.

5. **Results saving**: Storing the processed data for further use.



## Program Parameter Configuration

### 1. Selecting Assets

*	*`TICKERS`:* A list of asset symbols to be analyzed. These names will also be used to reference the assets later in the analysis.
	*	**Example**:
		```py
		TICKERS = ["BBAR", "BMA", "VALO"]
		```


*	*`TICKERS_YF`:* A list of Yahoo Finance codes for the selected `TICKERS`.
	*	**IMPORTANT**: The codes must be in the same order as the `TICKERS`.
	*	**Example**:
		```py
		TICKERS_YF = ["BBAR.BA", "BMA.BA", "VALO.BA"]
		```


### 2. Date Range

*	*`START_DATE`:* The start date of the data series in `YYYY-MM-DD` format.
	*	**Example**:
		```py
		START_DATE = "2023-01-01"
		```


*	*`END_DATE`:* The end date of the data series in `YYYY-MM-DD` format.
	*	If left **empty** (`""`), it will a**utomatically use the current date**.
	*	**Example**:
		```py
		END_DATE = "2024-12-31"
		```
		Or to fetch data up to today
		```py
		END_DATE = ""
		```


### 3. Data Export Settings

*	*`EXPORT_DATA`:* Specifies whether to export the data to **.csv** and **.xlsx files**. 
	*	Possible values:
		*	`True`: Exports the files.
		*	`False`: Does not export the files.
	*	**Example**:
		```py
		EXPORT_DATA = True
		```


*	*`OUTPUT_NAME_1`:* Filename for saving the ***adjusted closing prices (Adj Close)*** of the tickers.
	*	**Example**:
		```py
		OUTPUT_NAME_1 = "adjusted_prices"
		```


*	*`OUTPUT_NAME_2`:* Filename for also saving the ***calculated returns and volatility***.
	*	**Example**:
		```py
		OUTPUT_NAME_2 = "returns_volatility"
		```


### Complete Configuration Example

```py
TICKERS = ["BBAR", "BMA", "VALO"]
TICKERS_YF = ["BBAR.BA", "BMA.BA", "VALO.BA"]
START_DATE = "2023-01-01"
END_DATE = ""
EXPORT_DATA = True
OUTPUT_NAME_1 = "adjusted_prices"
OUTPUT_NAME_2 = "returns_volatility"
```


***************************



# 1. Kick-off: Libraries Importing, Variables Setup and Functions

In [1]:
import numpy as np
import pandas as pd
import time
import datetime as dt
from datetime import datetime

import yfinance as yf

import warnings

Setting the parameters:

In [2]:
warnings.simplefilter("ignore")
# Medimos el tiempo de ejecucion del programa
star_time = time.time()

MARKET_DAYS_YEAR = 252
OUTPUT_NAME_1 = 'historical-Adj_prices-byma'
OUTPUT_NAME_2 = 'historical-Adj_prices_plus-byma'
EXPORT_DATA = True

TICKERS = ['Index', 'ALUA', 'BBAR', 'BMA', 'BYMA', 'CEPU', 'COME', 'CRES', 'CVH', 'EDN', 'GGAL', 'LOMA', 'MIRG', 'PAMP', 'SUPV', 'TECO2', 'TGNO4', 'TGSU2', 'TRAN', 'TXAR', 'VALO', 'YPFD']

TICKERS_YF = ['^MERV', 'ALUA.BA', 'BBAR.BA', 'BMA.BA', 'BYMA.BA', 'CEPU.BA', 'COME.BA', 'CRES.BA', 'CVH.BA', 'EDN.BA', 'GGAL.BA', 'LOMA.BA', 'MIRG.BA', 'PAMP.BA', 'SUPV.BA', 'TECO2.BA', 'TGNO4.BA', 'TGSU2.BA', 'TRAN.BA', 'TXAR.BA', 'VALO.BA', 'YPFD.BA']

START_DATE = '2000-01-01'
END_DATE = ''


In [3]:
def ticker_simple_return():
	return list(map(lambda elem: elem + '_sr', tickers))

def ticker_log_return(): 
	return list(map(lambda elem: elem + '_lr', tickers))

def ticker_volat(): 
	return list(map(lambda elem: elem + '_v40', tickers))


In [4]:
def descargar_datos_yf(tickers, start_date=None, end_date=None, delay=1):
    if start_date is None:
        start_date = dt.datetime(2015, 1, 1)
    if end_date is None:
        end_date = dt.datetime.now()

    data_dict = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
            if not df.empty:
                data_dict[ticker] = df
                print(f'Descargado: {ticker}')
            else:
                print(f'Sin datos: {ticker}')
        except Exception as e:
            print(f'Error descargando {ticker}: {e}')
        time.sleep(delay)
	
    if data_dict:
        df = pd.concat(data_dict, axis=1)
    else:
        df = pd.DataFrame()

    return df

# 2. Data Loading

In [5]:
tickers = TICKERS
tickers_yf = TICKERS_YF

start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.now() if not END_DATE else datetime.strptime(END_DATE, "%Y-%m-%d")

raw_data = descargar_datos_yf(tickers_yf, start_date, end_date)
prices = raw_data.copy()

prices.index = pd.to_datetime(prices.index).strftime('%Y-%m-%d')

print('Index type:', prices.index.dtype)
prices.head()

Descargado: ^MERV
Descargado: ALUA.BA
Descargado: BBAR.BA
Descargado: BMA.BA
Descargado: BYMA.BA
Descargado: CEPU.BA
Descargado: COME.BA
Descargado: CRES.BA
Descargado: CVH.BA
Descargado: EDN.BA
Descargado: GGAL.BA
Descargado: LOMA.BA
Descargado: MIRG.BA
Descargado: PAMP.BA
Descargado: SUPV.BA
Descargado: TECO2.BA
Descargado: TGNO4.BA
Descargado: TGSU2.BA
Descargado: TRAN.BA
Descargado: TXAR.BA
Descargado: VALO.BA
Descargado: YPFD.BA
Index type: object


^MERV                                      ALUA.BA          \
Price      Adj Close  Close   High    Low   Open Volume Adj Close   Close   
Ticker         ^MERV  ^MERV  ^MERV  ^MERV  ^MERV  ^MERV   ALUA.BA ALUA.BA   
Date                                                                        
2000-01-03     552.0  552.0  560.0  550.0  550.0    0.0       NaN     NaN   
2000-01-04     523.0  523.0  551.0  523.0  551.0    0.0       NaN     NaN   
2000-01-05     533.0  533.0  534.0  517.0  523.0    0.0       NaN     NaN   
2000-01-06     529.0  529.0  534.0  526.0  533.0    0.0       NaN     NaN   
2000-01-07     522.0  522.0  533.0  521.0  529.0    0.0       NaN     NaN   

                            ... VALO.BA                            YPFD.BA  \
Price         High     Low  ...    High     Low    Open  Volume  Adj Close   
Ticker     ALUA.BA ALUA.BA  ... VALO.BA VALO.BA VALO.BA VALO.BA    YPFD.BA   
Date                        ...                                              
2000-01-03     NaN     NaN  ...     NaN     NaN     NaN     NaN  24.355097   
2000-01-04     NaN     NaN  ...     NaN     NaN     NaN     NaN  24.355097   
2000-01-05     NaN     NaN  ...     NaN     NaN     NaN     NaN  24.422564   
2000-01-06     NaN     NaN  ...     NaN     NaN     NaN     NaN  24.186430   
2000-01-07     NaN     NaN  ...     NaN     NaN     NaN     NaN  24.591227   

                                                                 
Price           Close       High        Low       Open   Volume  
Ticker        YPFD.BA    YPFD.BA    YPFD.BA    YPFD.BA  YPFD.BA  
Date                                                             
2000-01-03  36.099998  36.549999  36.049999  36.099998  24993.0  
2000-01-04  36.099998  36.099998  35.900002  36.099998   9896.0  
2000-01-05  36.200001  36.200001  36.099998  36.200001   2823.0  
2000-01-06  35.849998  36.099998  35.849998  35.849998   2285.0  
2000-01-07  36.450001  36.450001  36.000000  36.450001   4738.0  

[5 rows x 132 columns]

# 3. Data Cleaning

En esta estapa vamos a realizar lo siguiente: 
*	Modificamos los nombres de los Tickers, para mayor comodidad
*	Vamos a filtrar y quedarnos con los *Adj Close* de cada Ticker
*	Eliminamos los MultiIndex del DF
*	Agregamos un indice numerico creciente en la primer columna, a modo de numero de fila, en caso de que sea util en el futuro.
*	Tratamos los missing values y los valores negativos.
*	Hacemos un Checkpoint y guardamos lo realizado hasta el momento.

In [6]:
#prices = raw_data.copy()

In [7]:
for ticker_y, name in zip(tickers_yf, tickers):
	prices.rename(columns={ticker_y: name}, inplace=True)

df = prices
print("\nNiveles del índice en columnas:")
for i, name in enumerate(df.columns.names):
    print(f"Nivel {i} ({name}): {df.columns.unique(level=i).tolist()}")



Niveles del índice en columnas:
Nivel 0 (None): ['Index', 'ALUA', 'BBAR', 'BMA', 'BYMA', 'CEPU', 'COME', 'CRES', 'CVH', 'EDN', 'GGAL', 'LOMA', 'MIRG', 'PAMP', 'SUPV', 'TECO2', 'TGNO4', 'TGSU2', 'TRAN', 'TXAR', 'VALO', 'YPFD']
Nivel 1 (Price): ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
Nivel 2 (Ticker): ['Index', 'ALUA', 'BBAR', 'BMA', 'BYMA', 'CEPU', 'COME', 'CRES', 'CVH', 'EDN', 'GGAL', 'LOMA', 'MIRG', 'PAMP', 'SUPV', 'TECO2', 'TGNO4', 'TGSU2', 'TRAN', 'TXAR', 'VALO', 'YPFD']


In [8]:
# Filtro Adj Close de todos los tickers
prices = prices.xs('Adj Close', axis=1, level=1)

# Elimino multiindexes q no necesite
prices = prices.droplevel('Ticker', axis=1)

# Agregamos un indice "n" en la columna 1
prices.insert(0, 'n', 1, allow_duplicates=False)
prices['n'] = prices['n'].cumsum()

# Reemplazamos los missing values y negativos por 0
prices.fillna(0, inplace=True)
prices[prices < 0] = 0

prices

,n,Index,ALUA,BBAR,BMA,BYMA,CEPU,COME,CRES,CVH,...,MIRG,PAMP,SUPV,TECO2,TGNO4,TGSU2,TRAN,TXAR,VALO,YPFD
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1,552.0,0.0,4.108167,1.492395,0.0,0.0,0.095778,0.560380,0.0,...,1.214271,0.0,0.0,4.286654,0.0,0.816249,0.0,0.099021,0.0,24.355097
2000-01-04,2,523.0,0.0,3.907511,1.403398,0.0,0.0,0.087797,0.537507,0.0,...,1.165701,0.0,0.0,4.113706,0.0,0.798601,0.0,0.095631,0.0,24.355097
2000-01-05,3,533.0,0.0,3.960317,1.471857,0.0,0.0,0.089185,0.537507,0.0,...,1.165701,0.0,0.0,4.163120,0.0,0.794189,0.0,0.098536,0.0,24.422564
2000-01-06,4,529.0,0.0,3.981437,1.437627,0.0,0.0,0.090573,0.526071,0.0,...,1.165701,0.0,0.0,4.076645,0.0,0.794189,0.0,0.097810,0.0,24.186430
2000-01-07,5,522.0,0.0,3.823024,1.471857,0.0,0.0,0.089878,0.524927,0.0,...,1.165701,0.0,0.0,4.014878,0.0,0.807425,0.0,0.098294,0.0,24.591227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,6270,2348398.0,813.0,7840.000000,10725.000000,486.0,1575.0,196.000000,1320.000000,7410.0,...,24875.000000,3890.0,3420.0,2925.000000,3780.0,6560.000000,2420.0,772.000000,384.5,44650.000000
2025-02-25,6271,2282899.0,805.0,7610.000000,10500.000000,470.5,1495.0,187.750000,1285.000000,7760.0,...,24825.000000,3775.0,3290.0,2900.000000,3680.0,6310.000000,2315.0,753.000000,375.0,43150.000000
2025-02-26,6272,2275037.0,797.0,7640.000000,10625.000000,467.0,1465.0,181.750000,1275.000000,7350.0,...,24700.000000,3720.0,3320.0,2760.000000,3595.0,6210.000000,2260.0,737.000000,367.5,43000.000000


Creamos un Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifican con el sufijo *"v1"*:

In [9]:
# Checkpoint
df = prices
dataframes_string = ['prices']

for df_name in dataframes_string:
	df = globals()[df_name]
	globals()[f'{df_name}_v1'] = df.copy(deep=True)

# 4. Data Transformation

A cada activo le agregamos los siguientes calculos: 
* Rendimiento simple (o discreto), respecto de rueda anterior.
* Rendimiento logaritmico (o continuo), respecto de rueda anterior.
* Volatilidad (o desvio estandar) de las ultimas 40 ruedas, anualizado.

In [10]:
#prices = prices_v1.copy(deep=True)

In [11]:
# Insertamos los calculos de rendimientos y volatildiad
for i in range(len(tickers)):
	
	# Seleccionamos el ticker
	asset = tickers[i]

	# Buscamos la posicion de la columna del ticker
	pos = prices.columns.get_loc(asset)

	# Asignamos el nombre a la nueva col
	col_sr = ticker_simple_return()[i]
	col_lr = ticker_log_return()[i]
	col_v40 = ticker_volat()[i]

	# Insertamos una columna luego del ticker, con el nombre correspondiente al calculo y el ticker
	prices.insert(pos+1, col_sr, np.nan)
	prices.insert(pos+2, col_lr, np.nan)
	prices.insert(pos+3, col_v40, np.nan)

	# Calculamos los Simple Return diarios
	prices[col_sr] = (prices[asset] / prices[asset].shift(1)) - 1

	# Calculamos los Log Return diarios
	#prices[col_lr] = np.log(prices[asset] / prices[asset].shift(1))
	prices[col_lr] = np.log(prices[asset]).diff()

	# Calculo el Desvio St de las ultimas 40 ruedas anualizado
	prices[col_v40] = (prices[col_lr].rolling(window=40).std()) * np.sqrt(MARKET_DAYS_YEAR)

prices.fillna(0, inplace=True)

prices

,n,Index,Index_sr,Index_lr,Index_v40,ALUA,ALUA_sr,ALUA_lr,ALUA_v40,BBAR,...,TXAR_lr,TXAR_v40,VALO,VALO_sr,VALO_lr,VALO_v40,YPFD,YPFD_sr,YPFD_lr,YPFD_v40
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1,552.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,4.108167,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,24.355097,0.000000,0.000000,0.000000
2000-01-04,2,523.0,-0.052536,-0.053967,0.000000,0.0,0.000000,0.000000,0.000000,3.907511,...,-0.034829,0.000000,0.0,0.000000,0.000000,0.000000,24.355097,0.000000,0.000000,0.000000
2000-01-05,3,533.0,0.019120,0.018940,0.000000,0.0,0.000000,0.000000,0.000000,3.960317,...,0.029925,0.000000,0.0,0.000000,0.000000,0.000000,24.422564,0.002770,0.002766,0.000000
2000-01-06,4,529.0,-0.007505,-0.007533,0.000000,0.0,0.000000,0.000000,0.000000,3.981437,...,-0.007396,0.000000,0.0,0.000000,0.000000,0.000000,24.186430,-0.009669,-0.009716,0.000000
2000-01-07,5,522.0,-0.013233,-0.013321,0.000000,0.0,0.000000,0.000000,0.000000,3.823024,...,0.004937,0.000000,0.0,0.000000,0.000000,0.000000,24.591227,0.016737,0.016598,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,6270,2348398.0,-0.006822,-0.006845,0.468814,813.0,-0.013350,-0.013439,0.388031,7840.000000,...,-0.029358,0.341536,384.5,-0.010296,-0.010349,0.214927,44650.000000,-0.014349,-0.014453,0.397115
2025-02-25,6271,2282899.0,-0.027891,-0.028287,0.473114,805.0,-0.009840,-0.009889,0.387729,7610.000000,...,-0.024919,0.345148,375.0,-0.024707,-0.025018,0.205890,43150.000000,-0.033595,-0.034172,0.404333
2025-02-26,6272,2275037.0,-0.003444,-0.003450,0.471715,797.0,-0.009938,-0.009988,0.387414,7640.000000,...,-0.021477,0.347691,367.5,-0.020000,-0.020203,0.207394,43000.000000,-0.003476,-0.003482,0.404328


Creamos un Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifican con el sufijo *"v2"*:

In [12]:
# Checkpoint
df = prices
dataframes_string = ['prices']

for df_name in dataframes_string:
	df = globals()[df_name]
	globals()[f'{df_name}_v2'] = df.copy(deep=True)

# 5. Results Saving

Guardamos en formato .csv y .xlsx la serie de datos solo con cotizaciones y la serie de datos con cotizaciones y calculos de rendimientos y volatilidad.

In [13]:
if EXPORT_DATA:
	# Guardamos en un .csv
	prices_v1.to_csv('./' + OUTPUT_NAME_1 + '.csv')

	# Guardamos en un .xlsx
	with pd.ExcelWriter('./' + OUTPUT_NAME_1 + '.xlsx') as writer:
		prices_v1.to_excel(writer, sheet_name='Sheet1', index=True)


	# Guardamos en un .csv
	prices_v2.to_csv('./' + OUTPUT_NAME_2 + '.csv')

	# Guardamos en un .xlsx
	with pd.ExcelWriter('./' + OUTPUT_NAME_2 + '.xlsx') as writer:
		prices_v2.to_excel(writer, sheet_name='Sheet1', index=True)


	ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
	print(f'Data has already been exported. Date: {ahora}.')

else:
	ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
	print(f'No data has been exported. Date: {ahora}.')


end_time = time.time()
execution_time = end_time - star_time
print(f'\nExecution time: {round(execution_time, 2)} seconds.')

Data has already been exported. Date: 2025-03-03 21:49:27.

Execution time: 78.35 seconds.
